In [34]:
import os
import vtk
import numpy as np
from utilities import ReadVTPFile, WriteVTPFile
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

In [69]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU03A/PolarMap"
polar_map = os.path.join(InputFolder, "MyocardiumPolarMap.vtp")
PolarMap = ReadVTPFile(polar_map)

In [70]:
from sklearn.cluster import KMeans

MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
#MBFPolarValue = MBFPolarValue[(MBFPolarValue < 1000) & (MBFPolarValue > -100)]
X = MBFPolarValue.reshape(-1,1)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
labels = kmeans.labels_

/usr/local/anaconda3/envs/vtkfenics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [71]:
len(labels[labels==1])

410634

In [72]:
means = [MBFPolarValue[labels == i].mean() for i in range(2)]
mean_healthy = np.max(means)
mean_ischemic = np.min(means)
means

[116.96689, 70.072395]

In [73]:
ischemic_label = np.argmin(means)
ischemic_label

1

In [74]:
ischemia_mask = labels == ischemic_label
ischemia_mask

array([False, False, False, ...,  True,  True,  True])

In [75]:
ischemia_array = numpy_to_vtk(ischemia_mask.astype(np.uint8), deep=True)
ischemia_array.SetName("IsIschemic")
PolarMap.GetPointData().AddArray(ischemia_array)

ofile = os.path.join(InputFolder, "MyocardiumPolarMapIschemic.vtp")
WriteVTPFile(ofile, PolarMap)

In [76]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU03B/PolarMap"
polar_map = os.path.join(InputFolder, "MyocardiumPolarMap.vtp")
PolarMap = ReadVTPFile(polar_map)

In [77]:
from sklearn.mixture import GaussianMixture

MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
X = MBFPolarValue.reshape(-1,1)
gmm = GaussianMixture(n_components=2, means_init=np.array([[mean_ischemic], [mean_healthy]]))
gmm.fit(X)
labels = gmm.predict(X)



In [82]:
len(labels[labels == 0])

0

In [79]:
gmm.means_

array([[ 96.69513456],
       [139.47140924]])

In [83]:
gmm.weights_

array([0.01288013, 0.98711987])

In [84]:
np.unique(labels, return_counts=True)

(array([1]), array([767771]))